<a href="https://colab.research.google.com/github/ArneHei/Backend_Mobility/blob/main/Add_shipments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
#Test to see if code works

import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, Float, Boolean, Date, Time
from sqlalchemy.orm import sessionmaker, declarative_base
import pandas as pd
import sys
import os
import nbformat
import datetime
from google.colab import drive

drive.mount('/content/drive')
#"""



Mounted at /content/drive


In [ ]:
#Create data set from Filepath


# Define the base class for declarative models
Base = declarative_base()

class ShipmentDB(Base):
    __tablename__ = 'shipments'

    Shipment_ID = Column(String, primary_key=True)
    Transport = Column(String)
    Department = Column(String)
    Pickup_time = Column(String)
    Pickup_date = Column(Date)
    Delivery_time = Column(String)
    Delivery_date = Column(Date)
    Collection_Name = Column(String)
    Collection_City = Column(String)
    Collection_Address = Column(String)
    Collection_Postal_Code = Column(String)
    Delivery_Name = Column(String)
    Delivery_City = Column(String)
    Delivery_Address = Column(String)
    Delivery_Postal_Code = Column(String)
    Weight = Column(Float)
    Volume = Column(Float)
    Ldm = Column(Float)
    Content = Column(String)
    Units = Column(Integer)
    Unit_type = Column(String)
    Hazardous = Column(Boolean)

    def __repr__(self):
        return f"<Shipment(Shipment_ID='{self.Shipment_ID}', Transport='{self.Transport}', Department='{self.Department}')>"

#process new files to create shipments and add to DB
def process_shipment_files(file_paths: list):
    """Loads, processes, and filters shipment data from a list of Excel files.
    Returns a list of Shipment objects."""
    all_dfs = []
    for file_path in file_paths:
        try:
            df = pd.read_excel(file_path, header=0)
            all_dfs.append(df)
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
            return [] # Return empty list on error

    if not all_dfs:
        print("No dataframes were loaded.")
        return []

    concatenated_df = pd.concat(all_dfs, ignore_index=True)
    # Step 1: Create a working copy to avoid SettingWithCopyWarning
    df_modified = concatenated_df.copy()

    # Step 2: Initialize new columns with pd.NA
    df_modified['Pick up time'] = pd.NA
    df_modified['Delivery time'] = pd.NA

    # Step 3 & 4: Populate 'Delivery time' and 'Pick up time' based on 'Type'
    df_modified.loc[df_modified['Type'] == 'D', 'Delivery time'] = df_modified['Expected Handling']
    df_modified.loc[df_modified['Type'] == 'P', 'Pick up time'] = df_modified['Expected Handling']

    # Step 5: Define aggregation dictionary
    agg_dict = {
        col: (lambda x: str(x.dropna().iloc[0]) if not x.dropna().empty else pd.NA)
        if col in ['Delivery time', 'Pick up time']
        else 'first'
        for col in df_modified.columns if col != 'Shipment'
    }

    # Step 6: Group by 'Shipment' and apply aggregation
    df_modified = df_modified.groupby('Shipment', as_index=False).agg(agg_dict)

    # Filter for completed shipments (those with both 'Pick up time' and 'Delivery time')
    df_completed_shipments = df_modified[df_modified['Pick up time'].notna() & df_modified['Delivery time'].notna()]

    print("df_modified created and consolidated within the function.")

    # --- Start of new logic to create Shipment objects ---
    shipment_objects = []

    # Helper function to parse the custom datetime string 'YYYY-MM-DD HH:MM-HH:MM'
    def parse_custom_datetime_string(dt_value):
        if pd.notna(dt_value) and isinstance(dt_value, str):
            try:
                # Expected format: YYYY-MM-DD HH:MM-HH:MM
                parts = dt_value.split(' ')
                if len(parts) == 2:
                    date_part = parts[0] # YYYY-MM-DD
                    time_range_part = parts[1] # HH:MM-HH:MM

                    dt_obj_date = pd.to_datetime(date_part, utc=True)

                    return time_range_part, dt_obj_date.strftime('%Y-%m-%d')
                else:
                    # If not the specific format, try general parsing as fallback
                    try:
                        dt_obj = pd.to_datetime(dt_value, utc=True)
                        return dt_obj.strftime('%H:%M:%S'), dt_obj.strftime('%Y-%m-%d')
                    except:
                        return None, None # Could not parse
            except Exception as e:
                print(f"Warning: Could not parse custom datetime string '{dt_value}': {e}")
                return None, None
        elif pd.notna(dt_value) and isinstance(dt_value, pd.Timestamp):
            return dt_value.strftime('%H:%M:%S'), dt_value.strftime('%Y-%m-%d')
        return None, None # Handle NaN or other non-string/non-timestamp values

    for index, row in df_completed_shipments.iterrows():
        try:
            # a. Call the parse_custom_datetime_string helper function
            pickup_time_str, pickup_date_str = parse_custom_datetime_string(row['Pick up time'])
            # b. Similarly, call parse_custom_datetime_string for delivery
            delivery_time_str, delivery_date_str = parse_custom_datetime_string(row['Delivery time'])

            # c. Derive Department value
            current_shipment_id = row['Shipment']
            department_value = None
            # Use the original concatenated_df for department derivation
            p_type_entry = concatenated_df[(concatenated_df['Shipment'] == current_shipment_id) & (concatenated_df['Type'] == 'P')]
            if not p_type_entry.empty:
                derived_transport_id = p_type_entry.iloc[0]['Transport']
                if isinstance(derived_transport_id, str) and len(derived_transport_id) >= 3:
                    department_value = derived_transport_id[:3] + 'ST'

            # d. Convert Hazardous string to boolean
            hazardous_value = None
            if isinstance(row['Haz'], str):
                hazardous_value = True if row['Haz'].lower() == 'yes' else False
            elif isinstance(row['Haz'], bool):
                hazardous_value = row['Haz']

            # f. Create a Shipment object for each row
            shipment = Shipment(
                Shipment_ID=row['Shipment'],
                Transport=None, # As per instruction
                Department=department_value,
                Pickup_time=pickup_time_str,
                Pickup_date=pickup_date_str,
                Delivery_time=delivery_time_str,
                Delivery_date=delivery_date_str,
                Collection_Name=row['Collection Name'],
                Collection_City=row['Collection City'],
                Collection_Address=row['Collection Address'],
                Collection_Postal_Code=row['Collection Postal Code'],
                Delivery_Name=row['Delivery Name'],
                Delivery_City=row['Delivery City'],
                Delivery_Address=row['Delivery Address'],
                Delivery_Postal_Code=row['Delivery Postal Code'],
                Weight=row['Total Weight'],
                Volume=row['Total Volume'],
                Ldm=row['Ldm'],
                Content=row['Content'],
                Units= int(row['Expected Items']),
                Unit_type=row['Items of Type'],
                Hazardous=hazardous_value
            )
            # g. Append the newly created Shipment object to the shipment_objects list.
            shipment_objects.append(shipment)
        except Exception as e:
            print(f"Error creating Shipment object for row {index} (Shipment ID: {row['Shipment']}): {e}")

    print(f"Successfully created {len(shipment_objects)} Shipment objects.")
    # --- End of new logic ---
    engine = create_engine('sqlite:////content/drive/MyDrive/Driver app/shipments.db')

    # Create tables if they don't exist
    Base.metadata.create_all(engine)

    Session = sessionmaker(bind=engine)
    session = Session()

    for shipment in shipment_objects:
        try:
            # Check if a shipment with the same Shipment_ID already exists
            existing_shipment = session.query(ShipmentDB).filter_by(Shipment_ID=shipment.Shipment_ID).first()

            if existing_shipment is None:
                # Convert date strings to datetime.date objects
                pickup_date_obj = None
                if shipment.Pickup_date:
                    try:
                        pickup_date_obj = datetime.datetime.strptime(shipment.Pickup_date, '%Y-%m-%d').date()
                    except ValueError:
                        print(f"Warning: Could not parse Pickup_date '{shipment.Pickup_date}' for shipment {shipment.Shipment_ID}")

                delivery_date_obj = None
                if shipment.Delivery_date:
                    try:
                        delivery_date_obj = datetime.datetime.strptime(shipment.Delivery_date, '%Y-%m-%d').date()
                    except ValueError:
                        print(f"Warning: Could not parse Delivery_date '{shipment.Delivery_date}' for shipment {shipment.Shipment_ID}")


                db_shipment = ShipmentDB(
                    Shipment_ID=shipment.Shipment_ID,
                    Transport=shipment.Transport,
                    Department=shipment.Department,
                    Pickup_time=shipment.Pickup_time,
                    Pickup_date=pickup_date_obj,
                    Delivery_time=shipment.Delivery_time,
                    Delivery_date=delivery_date_obj,
                    Collection_Name=shipment.Collection_Name,
                    Collection_City=shipment.Collection_City,
                    Collection_Address=shipment.Collection_Address,
                    Collection_Postal_Code=shipment.Collection_Postal_Code,
                    Delivery_Name=shipment.Delivery_Name,
                    Delivery_City=shipment.Delivery_City,
                    Delivery_Address=shipment.Delivery_Address,
                    Delivery_Postal_Code=shipment.Delivery_Postal_Code,
                    Weight=shipment.Weight,
                    Volume=shipment.Volume,
                    Ldm=shipment.Ldm,
                    Content=shipment.Content,
                    Units=shipment.Units,
                    Unit_type=shipment.Unit_type,
                    Hazardous=shipment.Hazardous
                )
                session.add(db_shipment)
                print(f"Added shipment with Shipment ID: {shipment.Shipment_ID}")
            else:
                print(f"Shipment with Shipment ID: {shipment.Shipment_ID} already exists. Skipping.")

        except Exception as e:
            print(f"Error adding shipment to session: {e}")

    try:
        session.commit()
        print(f"Successfully committed changes to the database.")
    except Exception as e:
        session.rollback()
        print(f"Error committing shipments to database: {e}")
    finally:
        session.close()

    return shipment_objects

In [ ]:
"""
#Code to run to test

#DMP files for generating shipments.
file_paths = [
    "/content/drive/MyDrive/Colab Notebooks/Data handling/SEST0408.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/Data handling/SEST0508.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/Data handling/SEST0608.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/Data handling/SEST0708.xlsx",
    "/content/drive/MyDrive/Colab Notebooks/Data handling/SEST0808.xlsx"
]

#Calls the function and uses the selected data files to generate shipments from them
shipment_objects = process_shipment_files(file_paths)

print(f"Total shipment objects created: {len(shipment_objects)}")


# Displaying the first and 10th created shipment object for verification
if shipment_objects:
    print("\nDetails of the first created Shipment object:")
    first_shipment = shipment_objects[0]
    print(f"Shipment_ID: {first_shipment.Shipment_ID}")
    print(f"Transport: {first_shipment.Transport}")
    print(f"Department: {first_shipment.Department}")
    print(f"Pickup_time: {first_shipment.Pickup_time}")
    print(f"Pickup_date: {first_shipment.Pickup_date}")
    print(f"Delivery_time: {first_shipment.Delivery_time}")
    print(f"Delivery_date: {first_shipment.Delivery_date}")
    print(f"Collection_Name: {first_shipment.Collection_Name}")
    print(f"Units: {first_shipment.Units} {first_shipment.Unit_type} of {first_shipment.Content}")
    print("\n\nDetails of the 10th created Shipment object:")

    first_shipment = shipment_objects[9]
    print(f"Shipment_ID: {first_shipment.Shipment_ID}")
    print(f"Transport: {first_shipment.Transport}")
    print(f"Department: {first_shipment.Department}")
    print(f"Pickup_time: {first_shipment.Pickup_time}")
    print(f"Pickup_date: {first_shipment.Pickup_date}")
    print(f"Delivery_time: {first_shipment.Delivery_time}")
    print(f"Delivery_date: {first_shipment.Delivery_date}")
    print(f"Collection_Name: {first_shipment.Collection_Name}")
    print(f"Units: {first_shipment.Units} {first_shipment.Unit_type} of {first_shipment.Content}")
    """